In [ ]:
import scipy.io
import pandas as pd
import numpy as np 
from sklearn.model_selection import LeaveOneGroupOut, cross_val_score, cross_val_predict
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, roc_auc_score
import scipy.stats as stats
from mat_preproc import preproc

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set_style("darkgrid")

# LDA Projection onto Different Classifier

First, we hand coded the class attributes and each classifier listed on the literature.

In [ ]:
# class attributes
source_info = ["SC", "CR", "SI", "M", "FA"]
response_info = ["RS", "RO", "F", "MN", "SN"]

# the x-axis on the projection graph
x_axis = [(1, 1), (3, 1), (5, 1), (1, 2), (5, 2), (1, 3), (3, 3), (5, 3), (4, 4), (2, 4), (4, 5), (2, 5)]

x_name = np.array([f"{source_info[s-1]}-{response_info[r-1]}" for s, r in x_axis])
print(x_name)

In [ ]:
file_path = "data_CRM_SN_vs_MN_imbalLDA_order_proj_1.mat"

# in code, they did not leave out the two participant.
data_preproc = preproc(file_path, 1)
# combine SN and MN
pos1, neg1 = data_preproc.filter_index(2,5,2,4)
pos2, neg2 = data_preproc.filter_index(4,5,4,4)

pos_idx, neg_idx = data_preproc.merge_two_class(pos1, neg1, pos2, neg2)
X, y, subject = data_preproc.get_data_by_index(pos_idx, neg_idx, eliminate_trails=False)

logo = LeaveOneGroupOut()
LDA = LinearDiscriminantAnalysis(shrinkage = "auto", solver = 'eigen')

curr = 0
projections = []
for train_idx, test_idx in logo.split(X, y, subject):
    # select out the id of the participant that we left out
    participant = int(subject[curr])
    curr += len(test_idx)
    # this LOSO follows the sequence of ppl presented in subject
    X_train, X_test, y_train, y_test = X[train_idx], X[test_idx], y[train_idx], y[test_idx]
    LDA.fit(X_train, y_train)
    # project the whole data of this complete participant
    X_subject = data_preproc.get_data_by_participant(participant)
    projection = LDA.transform(X_subject)
    projections.append(projection)
projections = np.concatenate(projections)

In [ ]:
pos_idx = [10, 11]
neg_idx = [8, 9]

summary_stats = []

for source, response in x_axis:
    idx = data_preproc.filter_index_single_class(source, response, include_left_out=False)
    proj = projections[idx]
    mean = proj.mean()
    std = proj.std() / np.sqrt(len(proj)) * 1.96
    summary_stats.append((mean, std))

summary_stats = np.array(summary_stats)

plt.errorbar(x_name, summary_stats[:,0], yerr = summary_stats[:,1],
             fmt='o', capsize=3, c = 'black')
# pos
plt.errorbar(x_name[pos_idx], summary_stats[pos_idx,0],
             yerr = summary_stats[pos_idx,1] , fmt='o', capsize=3, c = 'lime', label="neg class")
# neg
plt.errorbar(x_name[neg_idx], summary_stats[neg_idx,0],
             yerr = summary_stats[neg_idx,1] , fmt='o', capsize=3, c = 'red', label="pos class")

_ = plt.xticks(rotation=45)
plt.legend()